# AR(2) Signal with Additive White Noise

Here we import nessessary packages 

In [1]:
using Polynomials
using DSP
using Plots
pyplot()

include("wiener_filter_scalar.jl")
include("wiener_filter_scalar_fft.jl")

wiener_filter_fft (generic function with 2 methods)

The parameters for this model are defined here.

In [2]:
# r, t = rand(), 100*rand()

# each need modulus less than one for the process to be stationary. 
# Also for the process to be rea-valued r1 and r2 are either real or complex conjugates
r = 2*rand(1) .- 1;

sig_v = 4
M_h = 50

50

## A Theoretical Solution

All the details for he formulation of `h_ana` are given in the Wiener Filtering Tutorial.

In [3]:
poly = complex(ones(5))

poly[1] = (r1*r2)/conj(r1*r2);
poly[2] = -(r1 + r2 + r1*r2*conj(r1+r2))/conj(r1*r2);
poly[3] = (sig_v^(-2) + 1 + abs(r1 + r2)^2 + abs(r1*r2)^2)/conj(r1*r2);
poly[4] = -(r1 + r2)/conj(r1*r2) - (r1+r2);
poly[5] = 1;
poly

rho_poly = Poly(poly)

rhos = roots(rho_poly)

rho1, rho2 = rhos[ abs.(rhos) .< 1]

phi0 = conj(rho1*rho2/r1/r2)/(sig_v)^2;

alpha(n) = sum([rho1^(n-k)*rho2^k for k = 0:n]);
beta(n) = sum([r1^(n-k)*r2^k for k = 0:n]);

M_gam = 1000 + M_h;

A = alpha.(0:M_gam);
B = beta.(0:M_gam);

gamma(n) = sum([conj(A[k])*B[k+n] for k = 1:1000]);

G = gamma.(0:M_h);

F = phi0*conv(G,A[1:M_h+1]);

h_ana = [F[1], F[2] - F[1]*(r1+r2)];

h_temp = [F[n] - (r1 + r2)*F[n-1] + r1*r2*F[n-2] for n = 3 : M_h];

h_ana = [h_ana; h_temp];

UndefVarError: UndefVarError: r1 not defined

## Data and Numerical Solution

In [4]:
l = [1, -(r1 + r2), r1*r2];

UndefVarError: UndefVarError: r1 not defined

Here we generate the data

In [5]:
steps = 1*10^7
discard = 10^4
steps_tot = steps + discard

noise_sig = randn(steps_tot)
sig = zeros(steps_tot);

sig[1:2] = noise_sig[1:2];
for i = 3 : steps_tot
    sig[i] = dot(-reverse(l)[1:2],sig[i - 2:i-1]) + noise_sig[i]
end
sig = sig[discard + 1 : steps_tot];

noise_pred = sig_v*randn(steps)
pred = sig .+ noise_pred;

UndefVarError: UndefVarError: l not defined

Now we call the program I wrote to Numerically solve the Weiner filter. 

In [6]:
h_num = wiener_filter_Nu(pred,sig, par=150);
h_num_fft = wiener_filter_fft(pred, sig,par = 150);

UndefVarError: UndefVarError: pred not defined

Now to verify the preformance.

In [7]:
sig_hat_num_fft = conv(real.(h_num_fft),pred)[1:steps]
sig_hat_num = conv(real.(h_num),pred)[1:steps]
sig_hat_ana = conv(real.(h_ana),pred)[1:steps]

view = 50
start = steps - view
tim = start:start + view;
error_num = sum((sig .- sig_hat_num).^2)/steps;
error_ana = sum((sig .- sig_hat_ana).^2)/steps;

print("error_num = ",error_num,"\n")
print("error_ana = ",error_ana)

plot(tim,[pred[tim] sig_hat_num[tim] sig_hat_num_fft[tim] sig_hat_ana[tim] sig[tim]],
    color=[:lightgrey :black :gray :red :orange ], 
    line=(2,[:dashdot :dash :dot :dot :solid ]),
    label=[ "pred" "sig_hat_num" "sig_hat_num_fft" "sig_hat_ana" "sig"])

UndefVarError: UndefVarError: h_num_fft not defined

In [8]:
error_num_fft = sig .-sig_hat_num_fft
error_num = sig .-sig_hat_num
error_ana = sig .-sig_hat_ana
lags = -5:10
C_ana = crosscov(error_ana,pred,lags)
C_num = crosscov(error_num,pred,lags)
C_num_fft = crosscov(error_num_fft,pred,lags)
plot(lags, [C_num C_num_fft C_ana],
    color=[:black :grey :red],
    line=(2,[:dash :dashdot :dot]),
    label=["sig_hat_num" "sig_hat_num_fft" "sig_hat_ana"],
    marker=([:hex :hex :d],6))

UndefVarError: UndefVarError: sig_hat_num_fft not defined

In [9]:
lags = -Nex:0
C_ana = crosscov(error_ana,pred,lags)
C_num = crosscov(error_num,pred,lags)
C_num_fft = crosscov(error_num_fft,pred,lags)

print(mean_and_var(C_num),'\n')
print(mean_and_var(C_num_fft),'\n')
print(mean_and_var(C_ana))

plot(lags, [C_num C_num_fft C_ana],
    color=[:black :grey :red],
    line=(2,[:dash :dashdot :dot]),
    label=["sig_hat_num" "sig_hat_num_fft" "sig_hat_ana"],
    marker=([:hex :hex :d],6))

UndefVarError: UndefVarError: Nex not defined

In [10]:
Nex = 1000;
Theta = 2*pi*(0:Nex - 1)/Nex
Z = exp.(im*Theta);

S_sig_hat_num = z_crossspect(sig_hat_num,sig_hat_num,70; win = "Bart")

H(z) = sum([h_ana[i+1]*z^(-i) for i = 0 : M_h-1]);
S_pred(z) = (conj(r1*r2)*rho_poly(z)/z^2)/((1-r1*z^(-1))*(1-r2*z^(-1))*(1-conj(r1)*z)*(1-conj(r2)*z));
S_sig_hat_ana(z) = H(z)*S_pred(z)*conj(H(conj(z)^(-1)));

plot(Theta,[real.(S_sig_hat_num.(Z)) real.(S_sig_hat_ana.(Z))],
    color=[:black :red],line=(2,[:dash :dot]),label=["S_sig_hat_num" "S_sig_hat_ana"])

UndefVarError: UndefVarError: sig_hat_num not defined